In [2]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

tf.enable_eager_execution()

In [3]:
tf.executing_eagerly()

True

In [5]:
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))

hello, [[4.]]


In [6]:
a = tf.constant([[1, 2], [3, 4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [7]:
b = tf.add(a, 1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [8]:
print(a*b)

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [9]:
import numpy as np
c = np.multiply(a, b)
print(c)

[[ 2  6]
 [12 20]]


In [10]:
print(a.numpy())

[[1 2]
 [3 4]]


In [11]:
def fizzbuzz(max_num):
    counter = tf.constant(0)
    max_num = tf.convert_to_tensor(max_num)
    for num in range(1, max_num.numpy() + 1):
        num = tf.constant(num)
        if int(num%3) == 0 and int(num%5) == 0:
            print("FizzBuzz")
        elif int(num%3) == 0:
            print("Fizz")
        elif int(num%5) == 0:
            print('Buzz')
        else:
            print(num.numpy())
        counter += 1
        
fizzbuzz(15)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz


In [12]:
class MySimpleLayer(tf.keras.layers.Layer):
    def __init__(self, output_units):
        super(MySimpleLayer, self).__init__()
        self.output_units = output_units
        
    def build(self, input_shape):
        self.kernel = self.add_variable(
            "kernel", [input_shape[-1], self.output_units])
    
    def call(self, input):
        return tf.matmul(input, self.kernel)

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_shape=(784,)),
    tf.keras.layers.Dense(10)])

In [14]:
class MNISTModel(tf.keras.Model):
    
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(units=10)
        self.dense2 = tf.keras.layers.Dense(units=10)
        
    def call(self, input):
        result = self.dense1(input)
        result = self.dense2(result)
        result = self.dense2(result)
        return result
    
model = MNISTModel()

In [16]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
    loss = w * w
    
grad = tape.gradient(loss, w)
print(grad)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


In [18]:
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (tf.cast(mnist_images[..., tf.newaxis]/255, tf.float32),
    tf.cast(mnist_labels, tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [19]:
mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3, 3], activation='relu'),
    tf.keras.layers.Conv2D(16, [3, 3], activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10)
])

In [20]:
for images, labels in dataset.take(1):
    print("Logits", mnist_model(images[0:1]).numpy())

Logits [[ 0.00356582 -0.0368358   0.01292696 -0.01591015 -0.01106439 -0.02059616
   0.00486663 -0.00373108  0.01239841 -0.00859677]]


In [21]:
optimizer = tf.train.AdamOptimizer()
loss_history = []

for (batch, (images, labels)) in enumerate(dataset.take(400)):
    if batch % 80 == 0:
        print()
    print('.', end='')
    with tf.GradientTape() as tape:
        logits = mnist_model(images, training=True)
        loss_value = tf.losses.sparse_softmax_cross_entropy(labels, logits)
        
    loss_history.append(loss_value.numpy())
    grads = tape.gradient(loss_value, mnist_model.variables)
    optimizer.apply_gradients(zip(grads, mnist_model.variables),
                             global_step=tf.train.get_or_create_global_step())


................................................................................
................................................................................
................................................................................
................................................................................
................................................................................

In [22]:
import matplotlib.pyplot as plt

plt.plot(loss_history)
plt.xlabel('Batch #')
plt.ylabel('Loss [entropy]')

Text(0, 0.5, 'Loss [entropy]')

In [23]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.W = tf.Variable(5., name='weight')
        self.B = tf.Variable(10., name='bias')
    def call(self, inputs):
        return inputs * self.W + self.B

NUM_EXAMPLES = 2000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs* 3 + 2 + noise

def loss(model, inputs, targets):
    error = model(inputs) - targets
    return tf.reduce_mean(tf.square(error))

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, [model.W, model.B])

model = Model()
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
print("Initial loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))

for i in range(300):
    grads = grad(model, training_inputs, training_outputs)
    optimizer.apply_gradients(zip(grads, [model.W, model.B]),
                             global_step=tf.train.get_or_create_global_step())
    if i % 20 == 0:
        print("Loss at step {:03d}: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))

print("Final loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))
print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))

Initial loss: 68.797
Loss at step 000: 66.114
Loss at step 020: 30.031
Loss at step 040: 13.940
Loss at step 060: 6.765
Loss at step 080: 3.565
Loss at step 100: 2.138
Loss at step 120: 1.501
Loss at step 140: 1.217
Loss at step 160: 1.091
Loss at step 180: 1.034
Loss at step 200: 1.009
Loss at step 220: 0.998
Loss at step 240: 0.993
Loss at step 260: 0.991
Loss at step 280: 0.990
Final loss: 0.989
W = 3.0115933418273926, B = 2.0261342525482178


In [24]:
x = tf.Variable(10.)
checkpoint = tf.train.Checkpoint(x=x)
x.assign(2.)
checkpoint_path = './ckpt/'
checkpoint.save('./ckpt/')

'./ckpt/-1'

In [25]:
x.assign(11.)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))
print(x)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>


In [27]:
import os

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3,3], activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10)
])
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
checkpoint_dir = './model_dir'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
root = tf.train.Checkpoint(optimizer=optimizer,
                          model=model,
                          optimizer_step=tf.train.get_or_create_global_step())
root.save(checkpoint_prefix)
root.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [28]:
tfe = tf.contrib.eager

m = tfe.metrics.Mean("loss")
m(0)
m(5)
m.result()
m([8, 9])
m.result()

<tf.Tensor: id=68222, shape=(), dtype=float64, numpy=5.5>

In [29]:
global_step = tf.train.get_or_create_global_step()
logdir = "./tb/"
writer = tf.contrib.summary.create_file_writer(logdir)
writer.set_as_default()

for _ in range(10):
    global_step.assign_add(1)
    with tf.contrib.summary.record_summaries_every_n_global_steps(100):
        tf.contrib.summary.scalar('global_step', global_step)

In [30]:
ls tb/

events.out.tfevents.1549957927.sean-mp.local.v2


In [31]:
def line_search_step(fn, init_x, rate=1.0):
    with tf.GradientTape() as tape:
        tape.watch(init_x)
        value = fn(init_x)
    grad = tape.gradient(value, init_x)
    grad_norm = tf.reduce_sum(grad * grad)
    init_value = value
    while value > init_value - rate * grad_norm:
        x = init_x - rate * grad
        value = fn(x)
        rate /= 2.0
    return x, value

In [32]:
def square(x):
    return tf.multiply(x, x)

grad = tfe.gradients_function(square)

In [33]:
square(3.).numpy()

9.0

In [34]:
grad(3.)[0].numpy()

6.0

In [35]:
gradgrad = tfe.gradients_function(lambda x: grad(x)[0])
gradgrad(3.)[0].numpy()

2.0

In [36]:
gradgradgrad = tfe.gradients_function(lambda x:gradgrad(x)[0])
gradgradgrad(3.)

[None]

In [37]:
def abs(x):
    return x if x > 0. else -x
grad = tfe.gradients_function(abs)

In [38]:
grad(3.)[0].numpy()

1.0

In [39]:
grad(-3.)[0].numpy()

-1.0

In [40]:
@tf.custom_gradient
def clip_gradient_by_norm(x, norm):
    y = tf.identity(x)
    def grad_fun(dresult):
        return [tf.clip_by_norm(dresult, norm), None]
    return y, grad_fn

In [41]:
def log1pexp(x):
    return tf.log(1 + tf.exp(x))
grad_log1exp = tfe.gradients_function(log1pexp)

In [42]:
grad_log1exp(0.)[0].numpy()

0.5

In [43]:
grad_log1exp(100.)[0].numpy()

nan

In [44]:
@tf.custom_gradient
def log1pexp(x):
    e = tf.exp(x)
    def grad(dy):
        return dy * (1 - 1 / (1 + e))
    return tf.log(1 + e), grad
grad_log1exp = tfe.gradients_function(log1pexp)

In [45]:
grad_log1exp(0.)[0].numpy()

0.5

In [46]:
grad_log1exp(100.)[0].numpy()

1.0

In [47]:
import time

def measure(x, steps):
    tf.matmul(x, x)
    start = time.time()
    for i in range(steps):
        x = tf.matmul(x, x)
    _ = x.numpy()
    end = time.time()
    return end - start

shape = (1000, 1000)
steps = 200
print("Time to multiply a {} matrix by itself {} times:".format(shape, steps))

with tf.device("/cpu:0"):
    print("CPU: {} secs".format(measure(tf.random_normal(shape), steps)))
    
if tfe.num_gpus() > 0:
    with tf.device("/gpu:0"):
        print("GPU: {} secs".format(measure(tf.random_normal(shape), steps)))
else:
    print("GPU: not found")

Time to multiply a (1000, 1000) matrix by itself 200 times:
CPU: 3.657733917236328 secs
GPU: not found


In [48]:
def my_py_func(x):
    x = tf.matmul(x, x)
    print(x)
    return x

with tf.Session() as sess:
    x = tf.placeholder(dtype=tf.float32)
    pf = tfe.py_func(my_py_func, [x], tf.float32)
    sess.run(pf, feed_dict={x: [[2.0]]})

tf.Tensor([[4.]], shape=(1, 1), dtype=float32)
